# Prepare data for MATSim simulation

In [2]:
%load_ext autoreload
%autoreload 2
%cd D:\sysmo-data-pt
%matplotlib inline

D:\sysmo-data-pt


In [3]:
# Load libs
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import matsim
import gzip
import numpy as np
import math
from tqdm import tqdm

## 1. Network processing using Java code
This is done by [pt2matsim](https://github.com/matsim-org/pt2matsim).

In [ ]:
"""
java -cp C:\matsim-13.0\pt2matsim-23.4-shaded.jar org.matsim.pt2matsim.run.CreateDefaultOsmConfig C:\sysmo-data-pt\dbs\defaultConfigFile.xml

java -cp C:\matsim-13.0\pt2matsim-23.4-shaded.jar org.matsim.pt2matsim.run.Osm2MultimodalNetwork C:\sysmo-data-pt\dbs\defaultConfigFile.xml

C:\osmosis\osmosis-0.49.2\bin\osmosis.bat --read-pbf file="C:\sysmo-data-pt\dbs\geo\sweden-latest.osm.pbf" --bounding-polygon file="C:\sysmo-data-pt\dbs\geo\zones\skane.poly" --write-xml file="C:\sysmo-data-pt\dbs\geo\skane.osm"

java -cp C:\matsim-13.0\pt2matsim-23.4-shaded.jar org.matsim.pt2matsim.run.Gtfs2TransitSchedule "C:\sysmo-data-pt\dbs\gtfs_se" "20230907" "EPSG:3006" "C:\sysmo-data-pt\dbs\outputTransitSchedule.xml.gz" "C:\sysmo-data-pt\dbs\outputVehiclesFile.xml"

java -cp C:\matsim-13.0\pt2matsim-23.4-shaded.jar org.matsim.pt2matsim.run.CreateDefaultPTMapperConfig C:\sysmo-data-pt\dbs\defaultConfigFilePTMapper.xml

java -cp C:\matsim-13.0\pt2matsim-23.4-shaded.jar org.matsim.pt2matsim.run.PublicTransitMapper C:\sysmo-data-pt\dbs\defaultConfigFilePTMapper.xml

java -cp C:\matsim-13.0\pt2matsim-23.4-shaded.jar org.matsim.pt2matsim.run.CheckMappedSchedulePlausibility "C:\sysmo-data-pt\dbs\transitSchedule.xml.gz" "C:\sysmo-data-pt\dbs\network.xml.gz" "EPSG:3006" "C:\sysmo-data-pt\dbs\gtfs_se\plausibility" 
"""

## 2. Prepare agents

In [10]:
# h-Home, w-Work, o-Other, s-School
activity_purpose_dict = {1: 'home', 4: 'work', 10: 'other', 6: 'school'}
mode_dict = {'Car': 'car', 'CarPassenger': 'car', 'Bike': 'bike',
             'Walking': 'walk', 'PublicTransport': 'pt'}

In [4]:
df_pop = pd.read_pickle('dbs/sysmo/syn_pop_all.pkl')
df_plan = pd.read_pickle('dbs/sysmo/df_act_plan.pkl')
df_plan['act_purpose'] = df_plan['act_purpose'].map(activity_purpose_dict)

### 3.1 Prepare non-car agents
This is to get those agents who exclusively use transit or other active modes.
The agents who have any car trips in combination with the other modes have been simulated in the other project.

In [9]:
# Agents who use pt and other modes
agents_non_car_users = df_plan.loc[df_plan['mode'].isin(['Bike', 'Walking', 'PublicTransport']), 'PId'].unique()

# Agents who use car
agents_car_users = df_plan.loc[df_plan['mode'].isin(['Car', 'CarPassenger']), 'PId'].unique()

# Agents who use all modes
intersection = np.intersect1d(agents_non_car_users, agents_car_users)

# Agents who exclusively use pt and other modes (target for further simulation)
agents_non_car_users = np.setdiff1d(agents_non_car_users, intersection)
len(agents_non_car_users)

4215776

In [10]:
df_plan = df_plan.loc[df_plan.PId.isin(agents_non_car_users), :]

## 3. Generate all agents' plans
The agents are divided into 9 batches.

In [4]:
def data2xml(data):
    num_activities = len(data)
    # Modes between activities
    mode_array = data['mode'].values[1:]
    act_purpose_array = data['act_purpose'].values
    X_array = data['POINT_X_sweref99'].values
    Y_array = data['POINT_Y_sweref99'].values
    act_end_array = data['act_end'].values
    # Process the case where the last activity crosses 0
    if act_end_array[-2] < 3:
        act_purpose_array = np.concatenate((act_purpose_array[-2], act_purpose_array[:-1]), axis=None)
        X_array = np.concatenate((X_array[-2], X_array[:-1]), axis=None)
        Y_array = np.concatenate((Y_array[-2], Y_array[:-1]), axis=None)
        act_end_array = np.concatenate((act_end_array[-2], act_end_array[:-1]), axis=None)
        act_end_array[-1] = 23.99
    # Excluding those who 1) do not move (stay home),
    # 2) start and end locations are not the same
    if (num_activities > 2) & (act_purpose_array[0] == act_purpose_array[-1]):
        writer.start_person(person_id=data['PId'].values[0])
        writer.start_plan(selected=True)
        for i in range(num_activities):
            # The last activity is always home
            if i == num_activities - 1:
                writer.add_activity(type=act_purpose_array[i],
                                    x=X_array[i],
                                    y=Y_array[i])
            else:
                writer.add_activity(type=act_purpose_array[i],
                        x=X_array[i],
                        y=Y_array[i],
                        end_time=act_end_array[i]*3600)
                writer.add_leg(mode=mode_dict[mode_array[i]])
        writer.end_plan()
        writer.end_person()

In [6]:
batch_num = 10
np.random.seed(68)
grps_dict = dict(zip(df_plan['PId'].unique(), np.random.randint(0, batch_num, size=df_plan['PId'].nunique())))
df_plan.loc[:, 'batch'] = df_plan['PId'].map(grps_dict)

In [8]:
for batch in range(0, batch_num):
    with gzip.open(f"dbs/scenarios/sweden/plans_{batch}.xml.gz", 'wb+') as f_write:
        writer = matsim.writers.PopulationWriter(f_write)
        writer.start_population()
        tqdm.pandas()
        df_plan.loc[df_plan.batch == batch, :].groupby('PId').progress_apply(data2xml)
        writer.end_population()

100%|██████████| 1020033/1020033 [06:12<00:00, 2736.07it/s]


## 4. Check batch 6
[Guice/ErrorInCustomProvider]: IllegalArgumentException: There exists already a node with id = 10000027567.
Existing node: [id=10000027567][coord=[x=670886.9942285293 | y=6585346.708522646]][type=null][nof_inlinks=2][nof_outlinks=2]
Node to be added: [id=10000027567][coord=[x=670886.9942285293 | y=6585346.708522646]][type=null][nof_inlinks=0][nof_outlinks=0].
Node is not added to the network.

In [7]:
df6 = df_plan.loc[df_plan.batch == 6, :].copy()
df6.head()

,PId,act_id,joint_PId_actid,act_purpose,act_start,act_end,mode,building_ID,building_type,Deso,Reg_model,New_Zone_ID,POINT_X_sweref99,POINT_Y_sweref99,POINT_X,POINT_Y,batch
30,8,0,8;0,home,3.000000,6.669886,,4168782,130,0115A0040,Samm,3711514,682961.2147,6.615291e+06,18.245410,59.635495,6
31,8,1,8;1,work,7.806994,15.163802,Walking,4175112,699,0115A0040,Samm,3711514,679308.0771,6.617570e+06,18.182664,59.657514,6
32,8,2,8;2,home,16.300911,17.187140,Walking,4168782,130,0115A0040,Samm,3711514,682961.2147,6.615291e+06,18.245410,59.635495,6
33,8,3,8;3,other,18.324248,21.853972,Car,5268978,699,1980C1350,Samm,3898002,586943.3729,6.609584e+06,16.540987,59.615351,6
34,8,4,8;4,home,22.991081,3.000000,Car,4168782,130,0115A0040,Samm,3711514,682961.2147,6.615291e+06,18.245410,59.635495,6


### 4.1 Find the agents near the node with error
It is a node in the network near Solna train station.

In [13]:
x1, y1 = 670886.9942285293, 6585346.708522646
def euc_d(row):
    return math.sqrt((row['POINT_X_sweref99'] - x1) ** 2 + (row['POINT_Y_sweref99'] - y1) ** 2)

tqdm.pandas()
df6.loc[:, 'd'] = df6.progress_apply(lambda row: euc_d(row), axis=1)

100%|██████████| 4168512/4168512 [01:19<00:00, 52110.87it/s]


In [28]:
df6 = df6.sort_values(by='d', ascending=True)
df6.to_parquet('dbs/scenarios/plans_6.parquet', index=False)

### 4.2 Check distance threshold

In [5]:
df6 = pd.read_parquet('dbs/scenarios/plans_6.parquet')

In [6]:
df6.loc[df6['d'] <= 100, 'PId'].nunique()

1

In [21]:
df6.loc[df6['d'] <= 100, ['POINT_X', 'POINT_Y']]

,POINT_X,POINT_Y
13643353,18.008811,59.372342


In [12]:
batch = 6
d_threshold = 500
to_keep = list(df6.loc[df6['d'] < d_threshold, 'PId'].unique())

In [13]:
with gzip.open(f"dbs/scenarios/sweden/plans_{batch}_t.xml.gz", 'wb+') as f_write:
    writer = matsim.writers.PopulationWriter(f_write)
    writer.start_population()
    tqdm.pandas()
    df6.loc[df6.PId.isin(to_keep[:1000]), :].\
        sort_values(by=['PId', 'act_id'], ascending=True).\
        groupby('PId').progress_apply(data2xml)
    writer.end_population()

100%|██████████| 1000/1000 [00:00<00:00, 2666.67it/s]
